In [ ]:
!pip install --upgrade -q git+https://github.com/keras-team/keras-cv
!pip install --upgrade -q git+https://github.com/soumik12345/wandb-addons wandb
!pip install -q namex
!apt install python3.10-venv
!git clone https://github.com/keras-team/keras-core.git && cd keras-core
!python pip_build.py --install

In [ ]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import tensorflow as tf
import tensorflow_datasets as tfds

import keras_cv
from keras_cv import visualization

import keras_core as keras
from keras_core import ops

import numpy as np

import resource
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

import wandb
from wandb_addons.keras.detection import log_predictions_to_wandb

In [ ]:
wandb.init(project="keras-cv-callbacks", entity="geekyrakshit", job_type="detection")

config = wandb.config
config.model_name = "retinanet_resnet50_pascalvoc"
config.image_size = 640
config.class_ids = [
    "Aeroplane",
    "Bicycle",
    "Bird",
    "Boat",
    "Bottle",
    "Bus",
    "Car",
    "Cat",
    "Chair",
    "Cow",
    "Dining Table",
    "Dog",
    "Horse",
    "Motorbike",
    "Person",
    "Potted Plant",
    "Sheep",
    "Sofa",
    "Train",
    "Tvmonitor",
    "Total",
]

In [ ]:
pretrained_model = keras_cv.models.RetinaNet.from_preset(
    config.model_name, bounding_box_format="xywh"
)

In [ ]:
filepath_1 = keras.utils.get_file(origin="https://i.imgur.com/gCNcJJI.jpg")
filepath_2 = keras.utils.get_file(origin="https://i.imgur.com/M8LR4fz.png")

image_1 = keras.utils.load_img(filepath_1)
image_1 = np.array(image_1)
image_2 = keras.utils.load_img(filepath_2)
image_2 = np.array(image_2)

visualization.plot_image_gallery(
    np.array([image_1]),
    value_range=(0, 255),
    rows=1,
    cols=1,
    scale=5,
)
visualization.plot_image_gallery(
    np.array([image_2]),
    value_range=(0, 255),
    rows=1,
    cols=1,
    scale=5,
)

In [ ]:
inference_resizing = keras_cv.layers.Resizing(
    config.image_size,
    config.image_size,
    pad_to_aspect_ratio=True,
    bounding_box_format="xywh"
)
image_1_resized = inference_resizing([image_1])
image_2_resized = inference_resizing([image_2])
image_batch = ops.concatenate([image_1_resized, image_2_resized], axis=0)
image_batch.shape

In [ ]:
class_mapping = dict(zip(range(len(config.class_ids)), config.class_ids))

y_pred = pretrained_model.predict(image_batch)

log_predictions_to_wandb(image_batch, y_pred, class_mapping)

wandb.finish()